### Video Indexer API Samples (Python)

This notebook provides samples for the following operations in Video Indexer:   

(1) Get account details.   
(2) Upload a video from URL.   
(2A) Upload a video from local file.   
(3) Wait for the video to finish indexing.   
(4) Search for video and get insights.
(5) Use the Widgets API.   
   
Make sure you're logged-in with `az` to authenticate your account.   
   
Copy the `.env.example` file to a new file named `.env`, and update the values with your own account settings.

In [6]:
%pip install python-dotenv
%pip install azure-identity



Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 51.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
# imports
from dotenv import dotenv_values
from pprint import pprint

from VideoIndexerClient.Consts import Consts
from VideoIndexerClient.VideoIndexerClient import VideoIndexerClient

Define the following parameters based on your account settings, in case they are different or not defined as environment variables:

In [9]:
config = dotenv_values(".env")

AccountName = 'vidindexernew'
ResourceGroup = 'vid'
SubscriptionId = '4f8d96c5-d3db-413a-8e01-5e3728dd04b2'

ApiVersion = '2024-01-01'
ApiEndpoint = 'https://api.videoindexer.ai'
AzureResourceManager = 'https://management.azure.com'

# create and validate consts
consts = Consts(ApiVersion, ApiEndpoint, AzureResourceManager, AccountName, ResourceGroup, SubscriptionId)

In [10]:
# Authenticate

# create Video Indexer Client
client = VideoIndexerClient()

# Get access tokens (arm and Video Indexer account)
client.authenticate_async(consts)

DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource. Error: Unexpected response "{'error': 'invalid_request', 'error_description': 'Identity not found'}"
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure accoun

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable. The requested identity has not been assigned to this resource. Error: Unexpected response "{'error': 'invalid_request', 'error_description': 'Identity not found'}"
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authen

#### Sample 1 - Get Account Basic Details
Get account details, not required in most cases.

In [ ]:
client.get_account_async()

#### Sample 2 - Index a Video from URL
Upload a video from URL, do not wait for the index operation to complete.   
Define `videoUrl` and `ExcludedAI` first.   
`ExcludedAI` - A list of the AIs you would like to exclude in the format `['Faces', 'Labels', 'Emotions','ObservedPeople']`. Leave empty if you do not want to exclude any AIs. For more details see [here](https://api-portal.videoindexer.ai/api-details#api=Operations&operation=Upload-Video:~:text=AI%20to%20exclude%20when%20indexing%2C%20for%20example%20for%20sensitive%20scenarios.%20Options%20are%3A%20Face/Observed%20peopleEmotions/Labels%7D).

In [ ]:
VideoUrl = 'https://www.youtube.com/watch?v=v1desDduz5M'
ExcludedAI = []

video_id = client.upload_url_async('my-video-name', VideoUrl, ExcludedAI, False)

#### Sample 2A - Index a Video From File
Upload From Local File.   
Define `LocalVideoPath` first.

In [ ]:
LocalVideoPath = 'YOUR_LOCAL_VIDEO_FILE_PATH'

file_video_id = client.file_upload_async(LocalVideoPath, video_name=None, excluded_ai=ExcludedAI)

#### Sample 3 - Polling on Video Completion Event
Wait for the video index to finish (Polling method).

In [ ]:
client.wait_for_index_async(file_video_id)

#### Sample 4 - Get the Video insights
Get the video insights.

In [ ]:
insights = client.get_video_async(file_video_id)
pprint(insights)

#### Sample 5 - Widgets API

In [ ]:
client.get_insights_widgets_url_async(video_id, widget_type='Keywords')
client.get_player_widget_url_async(video_id)

#### Sample 6 - Prompt Content API

In [ ]:

prompt_content = client.get_prompt_content(file_video_id)
pprint(prompt_content)